In [45]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [46]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [62]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [56]:
query = \
"""
SELECT * FROM echo_features_labs; 
"""
el = pd.read_sql_query(query, con)
el.head()

,row_id,subject_id,hadm_id,icustay_id,lab_albumin_min,lab_albumin_max,lab_bicarbonate_min,lab_bicarbonate_max,lab_ckmb_min,lab_ckmb_max,...,lab_ntprobnp_min,lab_ntprobnp_max,lab_ph_min,lab_ph_max,lab_tropi_min,lab_tropi_max,lab_tropt_min,lab_tropt_max,lab_wbc_min,lab_wbc_max
0,59653,31038,174978,200492,NaN,NaN,16.0,23.0,4.6,4.6,...,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.02,6.8,20.3
1,59654,70150,156140,245920,4.1,4.1,23.0,30.0,NaN,NaN,...,4453.0,4453.0,7.39,7.39,NaN,NaN,NaN,NaN,8.2,10.9
2,59655,54190,188571,292743,3.1,3.4,21.0,30.0,NaN,NaN,...,NaN,NaN,7.42,7.47,NaN,NaN,NaN,NaN,4.9,10.1
3,59656,5771,185291,274698,2.2,3.0,22.0,33.0,NaN,NaN,...,NaN,NaN,7.49,7.49,NaN,NaN,0.23,0.26,8.6,13.3
4,59657,80030,100442,249621,2.6,3.8,14.0,33.0,1.0,2.6,...,NaN,NaN,7.28,7.52,NaN,NaN,0.02,0.12,6.5,22.1


In [41]:
el['labid'].value_counts()

51301    25376
50912    25082
50820    21656
50813    19320
50862    18018
50910    17889
50911    16954
50811    14889
50810    14889
51003    13943
50920     7789
50908     6352
50963     3348
51288     2227
50889     1946
51002     1740
51300      296
Name: labid, dtype: int64

In [30]:
el.loc[el['row_id'] == 67394]

,row_id,lab_creatinine_charttime,lab_creatinine_dt,lab_creatinine_valuenum,lab_creatinine_valueuom,lab_creatinine_flag
3581,67394,2142-03-12 03:55:00,-1 days +12:16:00,1.5,mg/dL,abnormal
3582,67394,2142-03-13 03:23:00,11:44:00,1.1,mg/dL,None


In [34]:
el.loc[el['row_id'] == 64997]

,row_id,lab_creatinine_charttime,lab_creatinine_dt,lab_creatinine_valuenum,lab_creatinine_valueuom,lab_creatinine_flag
2464,64997,2127-12-05 17:07:00,05:06:00,8.6,mg/dL,abnormal
2465,64997,2127-12-05 06:55:00,-1 days +18:54:00,8.3,mg/dL,abnormal


In [38]:
el['row_id'].value_counts().value_counts()

1    25082
Name: row_id, dtype: int64

This is is really awkward. Basically, if there are multiple labs that meet the minimum dt, then this happens. One possibility is take the mean or something, but the flags may be conflicting. 

In [57]:
(~el.isnull()).sum() / 25512.

row_id                 1.000000
subject_id             1.000000
hadm_id                1.000000
icustay_id             1.000000
lab_albumin_min        0.706217
lab_albumin_max        0.706217
lab_bicarbonate_min    0.993062
lab_bicarbonate_max    0.993062
lab_ckmb_min           0.248863
lab_ckmb_max           0.248863
lab_creatinine_min     0.983145
lab_creatinine_max     0.983145
lab_crp_min            0.072868
lab_crp_max            0.072868
lab_egfr_min           0.000000
lab_egfr_max           0.000000
lab_hematocrit_min     0.995218
lab_hematocrit_max     0.995218
lab_hemoglobin_min     0.994904
lab_hemoglobin_max     0.994904
lab_inr_min            0.949710
lab_inr_max            0.949710
lab_lactate_min        0.757291
lab_lactate_max        0.757291
lab_platelet_min       0.994708
lab_platelet_max       0.994708
lab_ntprobnp_min       0.128685
lab_ntprobnp_max       0.128685
lab_ph_min             0.848855
lab_ph_max             0.848855
lab_tropi_min          0.045351
lab_trop

In [60]:
query = \
"""
SELECT * FROM echo_features; 
"""
ef = pd.read_sql_query(query, con)
ef.shape

(25512, 65)

In [61]:
ef.head()

,icustay_id,hadm_id,subject_id,age_at_intime,gender,height,weight,bmi,ethnicity,insurance,...,lab_tropt_max,lab_wbc_min,lab_wbc_max,ps_vaso,diag_xc,age_filter,intime_to_echo,time_filter,after_rowid,before_rowid
0,217810,188268,82208,22331 days 14:54:57,F,59.0,136.0,27.465671,WHITE,Medicaid,...,0.07,0.9,13.4,False,True,True,1 days 00:19:03,True,NaN,NaN
1,283373,118185,15472,22485 days 03:47:00,M,NaN,NaN,NaN,WHITE,Private,...,NaN,5.3,16.1,False,True,True,1 days 06:11:00,True,59668.0,NaN
2,271833,130443,2961,30992 days 18:16:15,F,61.0,106.0,20.026337,WHITE,Medicare,...,NaN,11.5,32.2,False,False,True,0 days 18:59:45,True,NaN,NaN
3,243977,110364,7429,28547 days 05:32:07,F,64.0,150.0,25.744629,WHITE,Medicare,...,NaN,4.9,29.1,True,False,True,17 days 10:33:53,False,NaN,NaN
4,286508,124736,17513,22104 days 00:50:44,M,71.0,198.0,27.612378,WHITE,Private,...,NaN,5.1,7.6,False,True,True,0 days 09:29:16,True,NaN,NaN
